# Introduction


This version of the simulator is configured to interact by default with **OpenAI**. It is important to consider that you must have an **API key** to execute the prompts that generate the conversations.

The following releases of the simulator will include connection methods with LLMs from other companies such as Anthropic, Perplexity, among others.

# Install libraries

Install the following libraries.

In [1]:
!sudo apt install libenchant-2-2
!pip install network-simulator
!pip install openai

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common enchant-2 hunspell-en-us libaspell15
  libhunspell-1.7-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-2-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common enchant-2 hunspell-en-us libaspell15
  libenchant-2-2 libhunspell-1.7-0 libtext-iconv-perl
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,431 kB of archives.
After this operation, 5,501 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtext-iconv-perl amd64 1.7-7build3 [14.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libaspell15 amd64 0.60.8-4build1 [325 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 dict

Clone github repository associated with the simulator.

In [2]:
!git clone https://github.com/minsoos/network_simulator.git

Cloning into 'network_simulator'...
remote: Enumerating objects: 344, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 344 (delta 67), reused 72 (delta 48), pack-reused 233 (from 1)
Receiving objects: 100% (344/344), 731.64 KiB | 5.38 MiB/s, done.
Resolving deltas: 100% (175/175), done.


Import libraries.

In [3]:
import subprocess
import os
import time

from network_simulator import make_yml, make_network_agents_yml
from network_simulator import get_pivoted_data, get_type_agents
from network_simulator import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from network_simulator import transform_time
from network_simulator import correctness_prompt, correctness_percentage
from anytree import Node, RenderTree, LevelOrderIter
from network_simulator import Post
from network_simulator.gpt3_5 import send_prompt_openai as send_prompt

import openai

# Define the parameters for the simulation

In [4]:
folder = os.path.join('network_simulator/schema')
name_simulation = 'simulation_tutorial'

network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "neutral"}
    ],

}

INTERVALS = 100
parameters = {
    "default_state": "{}",
    "load_module": "schema",
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent",
    "environment_params": {
        "prob_neighbor_spread": 0.05,
        "prob_tv_spread": 0.05,
        "prob_neighbor_cure": 0.006,
        "prob_backsliding": 0.05,
        "prob_dead": 0.001,
        "prob_repost": 0.8,
        "mean_time_connection": 10,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": INTERVALS,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 20,
        "m": 5
    },
    "num_trials": 1
}

prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }


Create the .yml file that will be used by SOIL.

In [5]:
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )
for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

os.path.join(folder, f'{name_simulation}.yml')

yml_path = os.path.join(folder, f'{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

# Run SOIL

In [6]:
command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 3.3409204483032227


In [7]:
name_simulation = 'simulation_tutorial'

analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)
dict_agents = get_type_agents(analysis_path, sql_table_path)

# Define root message

In [8]:
NEWS = 'They dictate preventive detention for Pablo Mackenna after being involved in a traffic accident while intoxicated in Las Condes. '
NEWS_BODY = '''
According to the information being handled, Mackenna crashed an executive taxi on Avenida Presidente Errázuriz and Calle Sánchez Fontecilla, causing serious damage to the other vehicle and leaving one person injured. When performing the breathalyzer, he returned 1.27 grams of alcohol per liter of blood. "We have to look at the conduct of the accused and the way in which he puts the lives of third parties at risk, which actually happened today," said Judge Acevedo. She added that "he hit a taxi driver, that the vehicle is his source of work, therefore, he will be unable to work, in addition to the license he has... he also injured another person." . Likewise, she emphasized that Mackenna crossed the red light, which is why she "committed a traffic violation, in addition to driving while intoxicated." "He will agree to the request of the Prosecutor's Office, and preventive detention will be ordered," said the judge, specifying that an investigation period of 90 days was determined.
'''
SHOW_TREE = True

In [9]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [10]:
TIME_BEG = "10:00"
TIME_END = "23:00"

list_nodes = [root]
num_real_messages = 0
for i in range(len(data)):
    aux = data[i]
    owner = int(aux[0])
    step = int(aux[1])
    step = transform_time(step, TIME_BEG, TIME_END)
    id_message = int(aux[2])
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index]

    type_agent = dict_agents[owner]

    ### Specific for repost attribute
    if attr_dict["repost"] in ["0", "1"]:
        attr_dict["repost"] = bool(int(attr_dict["repost"]))
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")

    if not attr_dict["repost"]:
        num_real_messages += 1
    ###

    message = ''
    news_i = NEWS
    if type_agent == "wise":
        news_i += NEWS_BODY

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, type_agent=type_agent,
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=news_i,
                             **attr_dict
                             )
    )

# Create messages

In [11]:
API_KEY = "insert API KEY"

client = openai.OpenAI(api_key=API_KEY)

TEMPERATURE = 1.0
list_prompts = []
i_tot = 0
i = 0
num_llm_errors = 0
MAX_LLM_ERRORS = 15
t = time.time()
t_0 = t

time_list = [0 for _ in range(num_real_messages)]
for node_i in LevelOrderIter(root):
    i_tot += 1
    if node_i == root:
        continue
    elif node_i.repost:
        continue
    i += 1
    instructions, prompt = node_i.get_prompt(language='english',
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    error_llm = True
    while error_llm:
        try:
            answer = send_prompt(client, instructions, prompt, temp=TEMPERATURE, max_tokens=1000, timeout=40)
            error_llm = False
        except Exception as err:
            print(f"Error encountered (message {i}/{num_real_messages}):", err)
            print("Instrucciones:", instructions)
            print("prompt", prompt)
            time.sleep(2)
            num_llm_errors += 1
            error_llm = True
            if num_llm_errors >= MAX_LLM_ERRORS:
                print("Max Errors reached. Local break run")
                break
    if num_llm_errors >= MAX_LLM_ERRORS:
        print("Max Errors reached. Break run")
        break
    correctness = correctness_percentage(answer)
    while correctness >= 0.2:
        print("Correction")
        correction_prompt = correctness_prompt(node_i.news, answer)
        print(correction_prompt)
        answer = send_prompt(client, instructions, correction_prompt, temp=TEMPERATURE, max_tokens=1000, timeout=40)
        correctness = correctness_percentage(answer)

    node_i.set_message(answer)
    list_prompts.append((node_i.name, prompt))
    t_aux = time.time()
    print(f"TIME: {t_aux-t}. Iter tot: {i_tot}. n. message: {i}")
    time_list[i-1] = t
    t = t_aux

print("TOTAL TIME:", t-t_0)

TIME: 1.0158486366271973. Iter tot: 5. n. message: 1
TIME: 0.6690773963928223. Iter tot: 6. n. message: 2
TIME: 0.943396806716919. Iter tot: 8. n. message: 3
TIME: 0.9126379489898682. Iter tot: 9. n. message: 4
TIME: 0.7346704006195068. Iter tot: 11. n. message: 5
TIME: 0.8900735378265381. Iter tot: 12. n. message: 6
TIME: 0.7833116054534912. Iter tot: 14. n. message: 7
TIME: 0.7695755958557129. Iter tot: 18. n. message: 8
TIME: 0.8203415870666504. Iter tot: 19. n. message: 9
TIME: 0.7964117527008057. Iter tot: 20. n. message: 10
TIME: 0.8461728096008301. Iter tot: 21. n. message: 11
TIME: 0.806265115737915. Iter tot: 22. n. message: 12
TIME: 0.7984163761138916. Iter tot: 23. n. message: 13
TIME: 0.760814905166626. Iter tot: 24. n. message: 14
TIME: 0.7745134830474854. Iter tot: 25. n. message: 15
TIME: 0.8147408962249756. Iter tot: 29. n. message: 16
TIME: 0.887547492980957. Iter tot: 31. n. message: 17
TIME: 0.7775382995605469. Iter tot: 32. n. message: 18
TIME: 1.0147485733032227. I

# Show propagation cascade

In [12]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {node.message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner}<<{node.name}>> (t={node.step})({node.type_agent})({node.parent.name}) {message}")

NEWS: They dictate preventive detention for Pablo Mackenna after being involved in a traffic accident while intoxicated in Las Condes. 
├── 13<<1>> (t=10:28)(wise)(0) repost
│   ├── 19<<3>> (t=11:27)(dumb)(1) repost
│   │   └── 9<<11>> (t=13:46)(dumb)(3) 'Yeah, he definitely needs to face the consequences of his actions. Drinking and driving is a serious issue that puts lives at risk. Hopefully, this serves as a wake-up call for everyone to think twice before getting behind the wheel after drinking. 🚫🍺'
│   │       ├── 4<<14>> (t=14:38)(dumb)(11) repost
│   │       ├── 16<<15>> (t=14:36)(wise)(11) 'Absolutely, drinking and driving is never okay. 🚫🍺 It's alarming how one wrong decision can have such severe consequences. Let's all remember to prioritize safety on the roads and make responsible choices.'
│   │       │   ├── 4<<16>> (t=14:45)(dumb)(15) 'Yeah, totally agree! 🚫🍺 It's shocking how easily things can go wrong when driving under the influence. Safety first, always. Let's all be 

# Create dataframe with messages

In [13]:
from output_dataframe import create_output_df

In [14]:
df = create_output_df.get_dataframe(list_nodes)
df

,target_step,source_node,iteration,target_message,interaction,stance,repost,target_user,source_message,source_user,source_step,target_node
0,10:28,0,1,,support,agree,True,13,They dictate preventive detention for Pablo Ma...,0,0,0
1,11:10,0,2,,support,agree,True,16,They dictate preventive detention for Pablo Ma...,0,0,0
2,11:27,1,3,,support,agree,True,19,They dictate preventive detention for Pablo Ma...,13,10:28,0
3,11:29,1,4,That's what he deserves! No special treatment ...,support,agree,False,16,They dictate preventive detention for Pablo Ma...,13,10:28,4
4,12:54,1,5,"Well, serves him right! People like him need t...",support,agree,False,16,They dictate preventive detention for Pablo Ma...,13,10:28,5
...,...,...,...,...,...,...,...,...,...,...,...,...
83,21:55,51,84,I totally agree with you. Drinking and driving...,support,agree,False,15,"Absolutely, you're spot on. This guy needs to ...",17,18:08,84
84,22:24,78,85,I agree! It's frustrating seeing this happen i...,support,agree,False,14,No way! Are we really doing this again? It's 2...,7,21:33,85
85,22:36,30,86,"No way! Everyone makes mistakes, don't pretend...",deny,against,False,7,Totally agree! Driving drunk is reckless and r...,19,15:50,86
86,22:50,75,87,That's crazy! It's unbelievable how some still...,deny,against,False,7,Why are people still driving drunk in this day...,12,21:05,87
